In [ ]:
#default_exp prep_novasar

# Prep NovaSAR

> Data preparation for NovaSAR acquisitions. SLC to dB for Gamma0.

In [ ]:
#hide
%load_ext autoreload
%autoreload 2

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
import os
import time
from glob import glob
import shutil

from ard_workflows.prep_utils import sedas_client, sedas_download, sedas_extract, sedas_find_datasets
from ard_workflows.prep_utils import run_snap_command, setup_logging
from ard_workflows.prep_utils import convert2cog, s3_upload_dir

In [ ]:
import pandas as pd

In [ ]:
os.environ['NOVASAR_PROCESS'] = '/opt/notebooks/anc/novasar_slc_tf_tc.xml'

notes
- default dem of auto-downloading srtm 1-arc sec
- ML+TC+TF default (and currently only) workflow
- gamma0 and HH ONLY (need to parse polarisations into snap chain, won't generalise)
- explode sub-scenes to scenes
- no compression on cog
- copy original metadata, but not outmetadata

In [ ]:
#export
def prep_novasar(sedas_supplierId, inter_dir="/tmp/data/intermediate/", 
                  s3_bucket="public-eo-data", s3_dir="novasar_uk_test/"):
    """
    Prepare a Sedas-hosted NovaSAR .zip scene(/strip) by suppliedId into a flavour of ARD
    using SNAP and upload output COGs to Catapult object storage. 
    """
    try:
        snap_gpt = os.environ['SNAP_GPT']
        novasar_process = os.environ['NOVASAR_PROCESS']
        root = setup_logging()

        inter_dir = f"{inter_dir}{sedas_supplierId}_tmp/"
        os.makedirs(inter_dir, exist_ok=True)
        master_scene_name = sedas_supplierId
        down_zip = f"{inter_dir}{master_scene_name}.zip"
        master_scene_dir = f"{down_zip[:-4]}/"
        
        
        print(f"{time.strftime('%Y-%m-%d %H:%M:%S')} Preparing {master_scene_name} within {inter_dir}")
        # find & download
        sedas_scene_res = sedas_client().search_product(sedas_supplierId)[0]
        sedas_download([sedas_scene_res], inter_dir)
        sedas_extract(down_zip, master_scene_dir)

        scenes_metadata = glob(f"{inter_dir}**/*metadata.xml", recursive=True)
        
        for scene_metadata in scenes_metadata:
            
            scene_name = scene_metadata.split('/')[-2]
            print(scene_name)
            
            out_dim = f"{inter_dir}{scene_name}_ML_TC_TF.dim"
            out_dimdir = f"{out_dim[:-4]}.data/"
            print(out_dimdir)
            
            out_cog_dir = f"{out_dimdir.replace('.data','_cog')}"
            os.makedirs(out_cog_dir, exist_ok=True)
            print(out_cog_dir)
            
            cmd = [snap_gpt, novasar_process, f"-Pinput={scene_metadata}", f"-Poutput={out_dim}"]
            run_snap_command(cmd)
            
            scene_bands = glob(f"{out_dimdir}**/*.img", recursive=True)
            for band_path in scene_bands:
                out_cog = f"{out_cog_dir}{scene_name}_{os.path.basename(band_path)[:-4]}.tif"
                print(out_cog)
                convert2cog(band_path, out_cog, 1)
            
            shutil.copy(scene_metadata, f"{out_cog_dir}original_metadata.xml")
            shutil.copy(out_dim, f"{out_cog_dir}process_metadata.dim")
            
            # upload
            s3_upload_dir(out_cog_dir, s3_bucket, s3_dir)
        print(f"{time.strftime('%Y-%m-%d %H:%M:%S')} Prepared {master_scene_name} at {s3_dir}")
#         clean_up(inter_dir) 
    except Exception as e:
        print(f"{time.strftime('%Y-%m-%d %H:%M:%S')} Failed with {e}")    
#         clean_up(inter_dir)

In [ ]:
done = ['NovaSAR_01_14295_slc_200823_233848_HH']

In [ ]:
result = sedas_find_datasets("POLYGON((-13.670367818336414 62.73764682827376,3.0947689004135848 62.73764682827376,3.0947689004135848 49.333035369023754,-13.670367818336414 49.333035369023754,-13.670367818336414 62.73764682827376))", 
                             "2000-01-01T00:00:00Z", 
                             "2020-10-27T00:00:00Z",
                             "NovaSAR"
                            )
pd.DataFrame(result['products'])


,productId,supplierId,type,satelliteName,instrumentName,modeName,sensorType,sensorResolution,coordinatesWKT,start,...,area,aoiCoveragePercent,usefulAreaPercent,polarisation,productType,latency,ql,thumbnail,vendorSpecific,downloadUrl
0,8b3308d59e50811022980dd83595b7ea,NovaSAR_01_16359_grd_201025_231828_HH,ARCHIVE,NovaSAR-1,6m-Stripmap,F3,SAR,6.0,"POLYGON((1.929749 51.297417,1.65757 51.349038,...",2020-10-25T23:18:29Z,...,1.240146e+09,0.0,100.0,HH,GRD,Standard,https://geobrowsertest.satapps.org/archiveql/a...,https://sedasdmtest.satapps.org/qls/qlmgr.php?...,"{'property': 'vendorSpecific', 'Filehash': '6d...",https://sedasdmtest.satapps.org/datamgr/datamg...
1,005751e56c8744048cb3c28f10b5de48,NovaSAR_01_16359_slc_201025_231828_HH,ARCHIVE,NovaSAR-1,6m-Stripmap,F3,SAR,6.0,"POLYGON((1.930699 51.298686,1.658025 51.350401...",2020-10-25T23:18:29Z,...,5.129263e+08,0.0,100.0,HH,SLC,Standard,https://geobrowsertest.satapps.org/archiveql/a...,https://sedasdmtest.satapps.org/qls/qlmgr.php?...,"{'property': 'vendorSpecific', 'Filehash': 'e5...",https://sedasdmtest.satapps.org/datamgr/datamg...
2,dc6039d27c8168a4d8728f80f021ed0e,NovaSAR_01_16343_scd_201025_100728_HH,ARCHIVE,NovaSAR-1,20m-ScanSAR,I1-4,SAR,20.0,"POLYGON((-7.327612 51.945704,-6.642753 52.0838...",2020-10-25T10:07:29Z,...,2.452427e+09,0.0,100.0,HH,SCD,Standard,https://geobrowsertest.satapps.org/archiveql/a...,https://sedasdmtest.satapps.org/qls/qlmgr.php?...,"{'property': 'vendorSpecific', 'Filehash': '07...",https://sedasdmtest.satapps.org/datamgr/datamg...
3,c79a5b656156b88cd5edab70676f27d0,NovaSAR_01_16328_grd_201024_231245_HH,ARCHIVE,NovaSAR-1,6m-Stripmap,F1,SAR,6.0,"POLYGON((-1.882012 51.888391,-1.601727 51.8484...",2020-10-24T23:12:46Z,...,4.893804e+08,0.0,100.0,HH,GRD,Standard,https://geobrowsertest.satapps.org/archiveql/a...,https://sedasdmtest.satapps.org/qls/qlmgr.php?...,"{'property': 'vendorSpecific', 'Filehash': '3b...",https://sedasdmtest.satapps.org/datamgr/datamg...
4,5bd6293c2ffd4a63cf9b0373db3d636c,NovaSAR_01_16328_slc_201024_231245_HH,ARCHIVE,NovaSAR-1,6m-Stripmap,F1,SAR,6.0,"POLYGON((-1.881807 51.889758,-1.601037 51.8497...",2020-10-24T23:12:46Z,...,5.644068e+08,0.0,100.0,HH,SLC,Standard,https://geobrowsertest.satapps.org/archiveql/a...,https://sedasdmtest.satapps.org/qls/qlmgr.php?...,"{'property': 'vendorSpecific', 'Filehash': 'a8...",https://sedasdmtest.satapps.org/datamgr/datamg...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
396,1f5f48e9cc14133e1c2149757f4d7ae3,NovaSAR_01_8868_scd_191220_235108_HH,ARCHIVE,NovaSAR-1,20m-ScanSAR,H1-5,SAR,20.0,"POLYGON((-2.226299 59.422195,-3.059788 59.5821...",2019-12-20T23:51:09Z,...,1.005670e+10,1.0,100.0,HH,SCD,Standard,https://geobrowsertest.satapps.org/archiveql/a...,https://sedasdmtest.satapps.org/qls/qlmgr.php?...,"{'property': 'vendorSpecific', 'Filehash': '3a...",https://sedasdmtest.satapps.org/datamgr/datamg...
397,0840e7f1c9e9377547019a739aa430b8,NovaSAR_01_8846_scd_191219_103615_HH,ARCHIVE,NovaSAR-1,20m-ScanSAR,H1-5,SAR,20.0,"POLYGON((-7.511056 51.589988,-8.902553 51.3958...",2019-12-19T10:36:16Z,...,9.847858e+09,1.0,100.0,HH,SCD,Standard,https://geobrowsertest.satapps.org/archiveql/a...,https://sedasdmtest.satapps.org/qls/qlmgr.php?...,"{'property': 'vendorSpecific', 'Filehash': '62...",https://sedasdmtest.satapps.org/datamgr/datamg...
398,b54c5e99d709067f574d7c43dcdcabe0,NovaSAR_01_8727_scd_22_191212_095704_VV_HH_HV_4,ARCHIVE,NovaSAR-1,30m-3Pol-ScanSAR,MidA,SAR,30.0,"POLYGON((-6.741796 54.971063,-6.010241 55.1133...",2019-12-12T09:57:05Z,...,1.676575e+09,0.0,100.0,VV/HH/HV,SCD,Standard,https://geobrowsertest.satapps.org/archiveql/a...,https://sedasdmtest.satapps.org/qls/qlmgr.php?...,"{'property': 'vendorSpecific', 'Filehash': 'c1...",https://sedasdmtest.satapps.org/datamgr/datamg...
399,c37b950cef38aeedd5049ed49b1126c5,NovaSAR_01_8727_scd_22_191212_095657_VV_HH_HV_3,ARCHIVE,NovaSAR-1,30m-3Pol-ScanSAR,MidA,SAR,30.0,"POLYGON((-6.495051 54.55122,-5.770831 54.69217...",

In [ ]:
res = pd.DataFrame(result['products'])

res[res.productType == 'SLC']

,productId,supplierId,type,satelliteName,instrumentName,modeName,sensorType,sensorResolution,coordinatesWKT,start,...,area,aoiCoveragePercent,usefulAreaPercent,polarisation,productType,latency,ql,thumbnail,vendorSpecific,downloadUrl
1,005751e56c8744048cb3c28f10b5de48,NovaSAR_01_16359_slc_201025_231828_HH,ARCHIVE,NovaSAR-1,6m-Stripmap,F3,SAR,6.0,"POLYGON((1.930699 51.298686,1.658025 51.350401...",2020-10-25T23:18:29Z,...,5.129263e+08,0.0,100.0,HH,SLC,Standard,https://geobrowsertest.satapps.org/archiveql/a...,https://sedasdmtest.satapps.org/qls/qlmgr.php?...,"{'property': 'vendorSpecific', 'Filehash': 'e5...",https://sedasdmtest.satapps.org/datamgr/datamg...
4,5bd6293c2ffd4a63cf9b0373db3d636c,NovaSAR_01_16328_slc_201024_231245_HH,ARCHIVE,NovaSAR-1,6m-Stripmap,F1,SAR,6.0,"POLYGON((-1.881807 51.889758,-1.601037 51.8497...",2020-10-24T23:12:46Z,...,5.644068e+08,0.0,100.0,HH,SLC,Standard,https://geobrowsertest.satapps.org/archiveql/a...,https://sedasdmtest.satapps.org/qls/qlmgr.php?...,"{'property': 'vendorSpecific', 'Filehash': 'a8...",https://sedasdmtest.satapps.org/datamgr/datamg...
7,d19710357d9fcbc949600286100de7b4,NovaSAR_01_16231_slc_201021_225532_HH,ARCHIVE,NovaSAR-1,6m-Stripmap,F7,SAR,6.0,"POLYGON((1.418476 51.529169,1.698445 51.491299...",2020-10-21T22:55:33Z,...,6.123630e+08,0.0,100.0,HH,SLC,Standard,https://geobrowsertest.satapps.org/archiveql/a...,https://sedasdmtest.satapps.org/qls/qlmgr.php?...,"{'property': 'vendorSpecific', 'Filehash': 'a2...",https://sedasdmtest.satapps.org/datamgr/datamg...
9,1532628f93676e3de8fe19b8139cafa1,NovaSAR_01_15861_slc_201009_232225_VV,ARCHIVE,NovaSAR-1,6m-Stripmap,F2,SAR,6.0,"POLYGON((-4.623971 51.59883,-4.344839 51.55943...",2020-10-09T23:22:25Z,...,4.952167e+08,0.0,100.0,VV,SLC,Standard,https://geobrowsertest.satapps.org/archiveql/a...,https://sedasdmtest.satapps.org/qls/qlmgr.php?...,"{'property': 'vendorSpecific', 'Filehash': 'd4...",https://sedasdmtest.satapps.org/datamgr/datamg...
11,7089c949264262d25b5a8e8881465cb6,NovaSAR_01_15785_slc_201007_095918_HH,ARCHIVE,NovaSAR-1,6m-Stripmap,G7,SAR,6.0,"POLYGON((-4.535956 50.656215,-4.335914 50.6967...",2020-10-07T09:59:19Z,...,4.038991e+08,0.0,100.0,HH,SLC,Standard,https://geobrowsertest.satapps.org/archiveql/a...,https://sedasdmtest.satapps.org/qls/qlmgr.php?...,"{'property': 'vendorSpecific', 'Filehash': 'fe...",https://sedasdmtest.satapps.org/datamgr/datamg...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350,92826e5e4297ceb2544b793704f44dc3,NovaSAR_01_11109_slc_200410_232727_HH,ARCHIVE,NovaSAR-1,6m-Stripmap,F9,SAR,6.0,"POLYGON((0.127337 51.623769,-0.146315 51.67790...",2020-04-10T23:27:28Z,...,6.531617e+08,0.0,100.0,HH,SLC,Standard,https://geobrowsertest.satapps.org/archiveql/a...,https://sedasdmtest.satapps.org/qls/qlmgr.php?...,"{'property': 'vendorSpecific', 'Filehash': '37...",https://sedasdmtest.satapps.org/datamgr/datamg...
365,8914808c20aeedd69a2570dd174da9e9,NovaSAR_01_10387_slc_200307_101114_HH,ARCHIVE,NovaSAR-1,6m-Stripmap,G1,SAR,6.0,"POLYGON((-0.706368 50.36297,-0.977687 50.32551...",2020-03-07T10:11:15Z,...,5.934961e+08,0.0,100.0,HH,SLC,Standard,https://geobrowsertest.satapps.org/archiveql/a...,https://sedasdmtest.satapps.org/qls/qlmgr.php?...,"{'property': 'vendorSpecific', 'Filehash': '16...",https://sedasdmtest.satapps.org/datamgr/datamg...
373,1c75ecc9c8d90894ca62281b06948a3a,NovaSAR_01_10135_slc_200215_094557_HH,ARCHIVE,NovaSAR-1,6m-Stripmap,F5,SAR,6.0,"POLYGON((-0.933414 50.397584,-0.667054 50.4491...",2020-02-15T09:45:57Z,...,5.611157e+08,0.0,100.0,HH,SLC,Standard,https://geobrowsertest.satapps.org/archiveql/a...,https://sedasdmtest.satapps.org/qls/qlmgr.php?...,"{'property': 'vendorSpecific', 'Filehash': '4f...",https://sedasdmtest.satapps.org/datamgr/datamg...
382,7d7e03c57d7739baaccfc4770544f18a,NovaSAR_01_9669_slc_200130_095029_HH,ARCHIVE,NovaSAR-1,6m-Stripmap,G7,SAR,6.0,"POLYGON((-4.623299 53.646621,-4.41069 53.68954...",2020-01-30T09:50:30Z,...,4.038746e+08,0.0,100.0,H

In [ ]:
for supplierId in res[(res.productType == 'SLC') & (res.polarisation == 'HH')].supplierId.values[:]:
    
    if supplierId not in done:
        prep_novasar(supplierId)

2020-11-17 18:20:30 Preparing NovaSAR_01_16359_slc_201025_231828_HH within /tmp/data/intermediate/NovaSAR_01_16359_slc_201025_231828_HH_tmp/


DEBUG:sedas_api:successful login.
DEBUG:sedas_api:successful login.
DEBUG:sedas_api:successful login.
DEBUG:sedas_api:successful login.
DEBUG:sedas_api:successful login.
INFO:bulk_downloader:0 downloads pending, 0 downloads in progress, 0 requests pending
INFO:bulk_downloader:2 active threads, 0 restarted.


2020-11-17 18:20:31 Downloading


DEBUG:bulk_downloader:1 downloads pending
DEBUG:bulk_downloader:1 downloads pending
INFO:bulk_downloader:downloading NovaSAR_01_16359_slc_201025_231828_HH to /tmp/data/intermediate/NovaSAR_01_16359_slc_201025_231828_HH_tmp/NovaSAR_01_16359_slc_201025_231828_HH.zip
DEBUG:sedas_api:successful login.
INFO:bulk_downloader:0 downloads pending, 1 downloads in progress, 0 requests pending
INFO:bulk_downloader:2 active threads, 0 restarted.
INFO:bulk_downloader:0 downloads pending, 1 downloads in progress, 0 requests pending
INFO:bulk_downloader:2 active threads, 0 restarted.
INFO:bulk_downloader:0 downloads pending, 1 downloads in progress, 0 requests pending
INFO:bulk_downloader:2 active threads, 0 restarted.
INFO:bulk_downloader:0 downloads pending, 1 downloads in progress, 0 requests pending
INFO:bulk_downloader:2 active threads, 0 restarted.
INFO:bulk_downloader:0 downloads pending, 1 downloads in progress, 0 requests pending
INFO:bulk_downloader:2 active threads, 0 restarted.
INFO:bulk_d

2020-11-17 18:21:16 Downloaded
2020-11-17 18:21:16 Extracting /tmp/data/intermediate/NovaSAR_01_16359_slc_201025_231828_HH_tmp/NovaSAR_01_16359_slc_201025_231828_HH.zip


...

/tmp/data/intermediate/NovaSAR_01_9629_slc_200129_094345_HH_tmp/NovaSAR_01_9629_slc_11_200129_094350_HH_2_ML_TC_TF_cog/NovaSAR_01_9629_slc_11_200129_094350_HH_2_Gamma0_Intensity_HH_db.tif
2020-11-18 06:05:19 Starting conversion: /tmp/data/intermediate/NovaSAR_01_9629_slc_200129_094345_HH_tmp/NovaSAR_01_9629_slc_11_200129_094350_HH_2_ML_TC_TF.data/Gamma0_Intensity_HH_db.img.
2020-11-18 06:05:27 Conversion complete: True.


DEBUG:root:running ['/opt/snap/bin/gpt', '/opt/notebooks/anc/novasar_slc_tf_tc.xml', '-Pinput=/tmp/data/intermediate/NovaSAR_01_9629_slc_200129_094345_HH_tmp/NovaSAR_01_9629_slc_200129_094345_HH/SV229/Image_Products$/NovaSAR_01_9629_slc_200129_094345_HH_/NovaSAR_01_9629_slc_11_200129_094354_HH_3/metadata.xml', '-Poutput=/tmp/data/intermediate/NovaSAR_01_9629_slc_200129_094345_HH_tmp/NovaSAR_01_9629_slc_11_200129_094354_HH_3_ML_TC_TF.dim']


NovaSAR_01_9629_slc_11_200129_094354_HH_3
/tmp/data/intermediate/NovaSAR_01_9629_slc_200129_094345_HH_tmp/NovaSAR_01_9629_slc_11_200129_094354_HH_3_ML_TC_TF.data/
/tmp/data/intermediate/NovaSAR_01_9629_slc_200129_094345_HH_tmp/NovaSAR_01_9629_slc_11_200129_094354_HH_3_ML_TC_TF_cog/


INFO:snap_stderr:stderr:INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapters
INFO:snap_stderr:stderr:INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: Incompatible GDAL 3.1.4 found on system. Internal GDAL 3.0.0 from distribution will be used.
INFO:snap_stderr:stderr:INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: Internal GDAL 3.0.0 set to be used by SNAP.
INFO:snap_stderr:stderr:INFO: org.esa.snap.core.util.EngineVersionCheckActivator: Please check regularly for new updates for the best SNAP experience.
INFO:snap_stderr:stderr:INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: Internal GDAL 3.0.0 set to be used by SNAP.
INFO:snap_stdout:Executing processing graph
INFO:snap_stderr:stderr:INFO: org.hsqldb.persist.Logger: dataFileCache open start
INFO:snap_stdout:....10%....20%....30%....40%....50%....60%....70%....80%....90% done.


/tmp/data/intermediate/NovaSAR_01_9629_slc_200129_094345_HH_tmp/NovaSAR_01_9629_slc_11_200129_094354_HH_3_ML_TC_TF_cog/NovaSAR_01_9629_slc_11_200129_094354_HH_3_Gamma0_Intensity_HH_db.tif
2020-11-18 06:11:34 Starting conversion: /tmp/data/intermediate/NovaSAR_01_9629_slc_200129_094345_HH_tmp/NovaSAR_01_9629_slc_11_200129_094354_HH_3_ML_TC_TF.data/Gamma0_Intensity_HH_db.img.
2020-11-18 06:11:45 Conversion complete: True.
2020-11-18 06:12:54 Prepared NovaSAR_01_9629_slc_200129_094345_HH at novasar_uk_test/


In [ ]:
prep_novasar('NovaSAR_01_14295_slc_200823_233848_HH')

DEBUG:sedas_api:successful login.


2020-11-17 16:25:03 Preparing NovaSAR_01_14295_slc_200823_233848_HH within /tmp/data/intermediate_test/NovaSAR_01_14295_slc_200823_233848_HH_tmp/


DEBUG:sedas_api:successful login.
DEBUG:sedas_api:successful login.
DEBUG:sedas_api:successful login.
DEBUG:sedas_api:successful login.
INFO:bulk_downloader:0 downloads pending, 0 downloads in progress, 0 requests pending
INFO:bulk_downloader:2 active threads, 0 restarted.


2020-11-17 16:25:04 Downloading


DEBUG:bulk_downloader:1 downloads pending
DEBUG:bulk_downloader:1 downloads pending
INFO:bulk_downloader:downloading NovaSAR_01_14295_slc_200823_233848_HH to /tmp/data/intermediate_test/NovaSAR_01_14295_slc_200823_233848_HH_tmp/NovaSAR_01_14295_slc_200823_233848_HH.zip
DEBUG:sedas_api:successful login.
INFO:bulk_downloader:0 downloads pending, 1 downloads in progress, 0 requests pending
INFO:bulk_downloader:2 active threads, 0 restarted.
INFO:bulk_downloader:0 downloads pending, 1 downloads in progress, 0 requests pending
INFO:bulk_downloader:2 active threads, 0 restarted.
INFO:bulk_downloader:0 downloads pending, 1 downloads in progress, 0 requests pending
INFO:bulk_downloader:2 active threads, 0 restarted.
INFO:bulk_downloader:0 downloads pending, 1 downloads in progress, 0 requests pending
INFO:bulk_downloader:2 active threads, 0 restarted.
INFO:bulk_downloader:0 downloads pending, 1 downloads in progress, 0 requests pending
INFO:bulk_downloader:2 active threads, 0 restarted.
INFO:b

...

/tmp/data/intermediate_test/NovaSAR_01_14295_slc_200823_233848_HH_tmp/NovaSAR_01_14295_slc_11_200823_233912_HH_7_ML_TC_TF_cog/NovaSAR_01_14295_slc_11_200823_233912_HH_7_Intensity_HH_db.tif
2020-11-17 16:34:16 Starting conversion: /tmp/data/intermediate_test/NovaSAR_01_14295_slc_200823_233848_HH_tmp/NovaSAR_01_14295_slc_11_200823_233912_HH_7_ML_TC_TF.data/Intensity_HH_db.img.
2020-11-17 16:34:22 Conversion complete: True.


DEBUG:root:running ['/opt/snap/bin/gpt', '/opt/notebooks/anc/novasar_slc_tc.xml', '-Pinput=/tmp/data/intermediate_test/NovaSAR_01_14295_slc_200823_233848_HH_tmp/NovaSAR_01_14295_slc_200823_233848_HH/SV229/Image_Products$/NovaSAR_01_14295_slc_200823_233848_HH_/NovaSAR_01_14295_slc_11_200823_233916_HH_8/metadata.xml', '-Poutput=/tmp/data/intermediate_test/NovaSAR_01_14295_slc_200823_233848_HH_tmp/NovaSAR_01_14295_slc_11_200823_233916_HH_8_ML_TC_TF.dim']


NovaSAR_01_14295_slc_11_200823_233916_HH_8
/tmp/data/intermediate_test/NovaSAR_01_14295_slc_200823_233848_HH_tmp/NovaSAR_01_14295_slc_11_200823_233916_HH_8_ML_TC_TF.data/
/tmp/data/intermediate_test/NovaSAR_01_14295_slc_200823_233848_HH_tmp/NovaSAR_01_14295_slc_11_200823_233916_HH_8_ML_TC_TF_cog/


INFO:snap_stderr:stderr:INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapters
INFO:snap_stderr:stderr:INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: Incompatible GDAL 3.1.4 found on system. Internal GDAL 3.0.0 from distribution will be used.
INFO:snap_stderr:stderr:INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: Internal GDAL 3.0.0 set to be used by SNAP.
INFO:snap_stderr:stderr:INFO: org.esa.snap.core.util.EngineVersionCheckActivator: Please check regularly for new updates for the best SNAP experience.
INFO:snap_stderr:stderr:INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: Internal GDAL 3.0.0 set to be used by SNAP.
INFO:snap_stdout:Executing processing graph
INFO:snap_stderr:stderr:INFO: org.hsqldb.persist.Logger: dataFileCache open start
INFO:snap_stderr:stderr:INFO: org.esa.snap.core.dataop.dem.ElevationFile: http retrieving http://step.esa.int/auxdata/dem/SRTMGL1/N50W003.SRTMGL1.hgt.zip
INFO:snap_stdout:....10%....20%....30%....40%....50%....

/tmp/data/intermediate_test/NovaSAR_01_14295_slc_200823_233848_HH_tmp/NovaSAR_01_14295_slc_11_200823_233916_HH_8_ML_TC_TF_cog/NovaSAR_01_14295_slc_11_200823_233916_HH_8_Intensity_HH_db.tif
2020-11-17 16:35:27 Starting conversion: /tmp/data/intermediate_test/NovaSAR_01_14295_slc_200823_233848_HH_tmp/NovaSAR_01_14295_slc_11_200823_233916_HH_8_ML_TC_TF.data/Intensity_HH_db.img.
2020-11-17 16:35:39 Conversion complete: True.
2020-11-17 16:36:47 Prepared NovaSAR_01_14295_slc_200823_233848_HH at novasar_uk_test/



## Export

In [ ]:
#hide
from nbdev.export import notebook2script; notebook2script()

Converted 1_prep_utils.ipynb.
Converted 2_prep_novasar.ipynb.
